In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
import pandas as pd

In [ ]:
df=pd.read_csv("/content/WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head(5)

FileNotFoundError: [Errno 2] No such file or directory: '/content/WA_Fn-UseC_-Telco-Customer-Churn.csv'

In [ ]:
df.drop("customerID",axis="columns", inplace=True)

In [ ]:
df.dtypes

In [ ]:
pd.to_numeric(df.TotalCharges,errors="coerce").isnull()

In [ ]:
df=df[df.TotalCharges!=" "]
df.shape

In [ ]:
df["TotalCharges"]=pd.to_numeric(df.TotalCharges)

In [ ]:
x=df[df.Churn=="No"].tenure
y=df[df.Churn=="Yes"].tenure
plt.xlabel("Tenure")
plt.xlabel("Customer Count")
plt.title("Customer Churn Analysis")
plt.hist([y,x],color=["green","red"],label=["Yes","No"])
plt.legend()

In [ ]:
x=df[df.Churn=="No"].MonthlyCharges
y=df[df.Churn=="Yes"].MonthlyCharges
plt.xlabel("MonthlyCharges")
plt.xlabel("Customer Count")
plt.title("Customer Churn Analysis")
plt.hist([y,x],color=["green","red"],label=["Yes","No"])
plt.legend()  # here the data shows us the data imbalence
# first i have predicted without consider ing the data imbalence
#i have also done data ajustment

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [ ]:
def uni(df):
  for i in df:
    print(f"{i} : {df[i].unique()}")

In [ ]:
uni(df)

In [ ]:
df.replace("No internet service","No",inplace=True)
df.replace("No phone service","No",inplace=True)

In [ ]:
list=["gender","Partner","Dependents","PhoneService","MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV","StreamingMovies","Contract","PaperlessBilling","PaymentMethod","Churn"]
for i in list:
  df[i]=le.fit_transform(df[i])

In [ ]:
df.sample(5)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc=MinMaxScaler()

In [ ]:
l2=["tenure","MonthlyCharges","TotalCharges"]
df[l2]=sc.fit_transform(df[l2])

In [ ]:
df.sample(5)

In [ ]:
from sklearn.model_selection import train_test_split as tts
x_train,x_test,y_train,y_test=tts(df.drop("Churn",axis="columns"),df.Churn,test_size=0.2,random_state=5)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape )

In [ ]:
model=keras.Sequential([
    keras.layers.Dense(19,input_shape=(19,),activation="relu"),
    keras.layers.Dense(10,activation="relu"),
    keras.layers.Dense(1,activation="sigmoid")
])

model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.fit(x_train,y_train,epochs=100)

In [ ]:
y_pre=model.predict(x_test)
y_pre[:5]

In [ ]:
y_pred=[]
for i in y_pre:
  if i>0.5:
    y_pred.append(1)
  else:
    y_pred.append(0)

In [ ]:
y_pred[:5]

In [ ]:
np.array(y_test[:5])

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
import seaborn as sns
cm=tf.math.confusion_matrix(labels=y_test,predictions=y_pred)

plt.figure(figsize=(10,7))
sns.heatmap(cm,annot=True,fmt="d")
plt.xlabel("Predicted")
plt.ylabel("Truth")

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_pred))

In [ ]:
def ANN(x_train,y_train,x_test,y_test,loss,weight):
  model=keras.Sequential([
    keras.layers.Dense(19,input_shape=(19,),activation="relu"),
    keras.layers.Dense(10,activation="relu"),
    keras.layers.Dense(1,activation="sigmoid")
  ])

  model.compile(optimizer="adam",
                loss=loss,
                metrics=["accuracy"])
  if(weight==-1):
    model.fit(x_train,y_train,epochs=100)
  else:
    model.fit(x_train,y_train,epochs=100,class_weight=weight)
  print("\n\n Evcaluation result:\n")
  print(model.evaluate(x_test,y_test))
  y_preds=model.predict(x_test)
  y_preds=np.round(y_preds)
  print("\nclassification report: ",classification_report(y_test,y_preds))
  return y_preds

In [ ]:
#ANN(x_train,y_train,x_test,y_test,"binary_crossentropy",-1)

Mitigating Skewdness of Data  Under sampling


In [ ]:
#class count
count_0,count_1=df.Churn.value_counts()
df_0=df[df.Churn==0]
df_1=df[df.Churn==1]

In [ ]:
df_0.shape,df_1.shape

In [ ]:
df_0_under=df_0.sample(count_1)
df_train_under=pd.concat([df_0_under,df_1],axis=0)
df_train_under.Churn.value_counts()

In [ ]:
x=df_train_under.drop("Churn",axis="columns")
y=df_train_under.Churn
x_tra,x_tes,y_tra,y_tes=tts(x,y,test_size=0.001,random_state=15,stratify=y)

In [ ]:
model_undersample=keras.Sequential([
    keras.layers.Dense(19,input_shape=(19,),activation="relu"),
    keras.layers.Dense(10,activation="relu"),
    keras.layers.Dense(1,activation="sigmoid")
])

model_undersample.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

model_undersample.fit(x_tra,y_tra,epochs=100)    # you can call the ANN instead of rewriting the steps again

In [ ]:
y_pr=model_undersample(x_test)
y_pr=np.round(y_pr).flatten()

In [ ]:
print(classification_report(y_test,y_pr))# this has improved a bit but i dont know this is right or not

2.Over Sampling

In [ ]:
df_1_over=df_1.sample(count_0,replace=True)
df_1_over.shape
df_over=pd.concat([df_1_over,df_0],axis=0)

In [ ]:
df_over.shape

In [ ]:
x_tr,_,y_tr,__=tts(df_over.drop("Churn",axis="columns"),df_over.Churn,test_size=0.0001,stratify=df_over.Churn)
x_tr.shape

In [ ]:
y_p=ANN(x_tr,y_tr,x_test,y_test,"binary_crossentropy",-1)

3.SMOTE Technique


In [ ]:
from imblearn.over_sampling import SMOTE
smote=SMOTE(sampling_strategy="minority")
x_sm,y_sm=smote.fit_resample(x_t,y_t)
y_sm.value_counts()

In [ ]:
x_t,_,y_t,__=tts(df.drop("Churn",axis="columns"),df.Churn,test_size=0.2,random_state=15,stratify=df.Churn)
x_t.shape

In [ ]:
y_p_smote=ANN(x_sm,y_sm,x_test,y_test,"binary_crossentropy",-1)

4.Ensemble with undersampling


In [ ]:
df.Churn.value_counts()

In [ ]:
x_tt,_,y_tt,__=tts(df.drop("Churn",axis="columns"),df.Churn,test_size=0.2,random_state=15,stratify=df.Churn)
x_t.shape

In [ ]:
df3=x_tt.copy()
df3["Churn"]=y_tt

In [ ]:
df3_0=df3[df3.Churn==0]
df3_1=df3[df3.Churn==1]

In [ ]:
def get_train_batch(df_max,df_min,st,end):
   df_tt=pd.concat([df_max[st:end],df_min],axis=0)
   x_trainn=df_tt.drop("Churn",axis="columns")
   y_trainn=df_tt.Churun
   return x_trainn,y_trainn

In [ ]:
x_triann,y_trainn=get_train_batch(df3_0,df3_1,0,1869)
y_pree1=ANN(x_trainn,y_trainn,x_test,y_test,"binary_crossentropy",-1)